In [0]:
SIZE =  (224, 244)
ROOT_PATH = '/content/drive/'

In [0]:
# if you wanna mount your own google drive
from google.colab import drive
drive.mount('/content/drive') 


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
%cd /content/drive/My\ Drive/practica_aidl/pipeline/recognition

/content/drive/My Drive/practica_aidl/pipeline/recognition


In [0]:
!ls  "/content/drive/My Drive/practica_aidl/backups"

BEST_checkpoint_vggsiamese_1.pth.tar  folds.good.tar.gz		WIDER_test.zip
cfp-dataset-changed-directory.tar.gz  joined_fddb_datasets.zip	WIDER_train.zip
examples			      wider_face_split.zip	WIDER_val.zip


In [0]:
from models import *
from torchvision import transforms
from PIL import Image
import torch

In [0]:

trans = transforms.Compose([transforms.Resize(SIZE), transforms.ToTensor()])
path_image_1 = ROOT_PATH + 'My Drive/practica_aidl/backups/examples/1.jpg'
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
cos = nn.CosineSimilarity(dim=1, eps=1e-6)



In [0]:
image = Image.open(path_image_1).convert('RGB')
image = trans(image)

In [0]:
image

tensor([[[0.9686, 0.9686, 0.9686,  ..., 0.1176, 0.1137, 0.1137],
         [0.9686, 0.9686, 0.9686,  ..., 0.1137, 0.1137, 0.1137],
         [0.9686, 0.9686, 0.9686,  ..., 0.1137, 0.1137, 0.1137],
         ...,
         [0.7294, 0.7569, 0.7961,  ..., 0.6706, 0.6902, 0.7059],
         [0.7059, 0.7412, 0.7882,  ..., 0.7176, 0.7490, 0.7765],
         [0.6863, 0.7216, 0.7647,  ..., 0.6980, 0.7569, 0.8000]],

        [[0.9608, 0.9608, 0.9608,  ..., 0.1137, 0.1098, 0.1098],
         [0.9608, 0.9608, 0.9608,  ..., 0.1098, 0.1098, 0.1098],
         [0.9608, 0.9608, 0.9608,  ..., 0.1098, 0.1098, 0.1098],
         ...,
         [0.7216, 0.7490, 0.7882,  ..., 0.6627, 0.6824, 0.6980],
         [0.6980, 0.7333, 0.7804,  ..., 0.7098, 0.7412, 0.7686],
         [0.6784, 0.7137, 0.7569,  ..., 0.6902, 0.7490, 0.7922]],

        [[0.9804, 0.9804, 0.9804,  ..., 0.1373, 0.1333, 0.1333],
         [0.9804, 0.9804, 0.9804,  ..., 0.1333, 0.1333, 0.1333],
         [0.9804, 0.9804, 0.9804,  ..., 0.1333, 0.1333, 0.

In [0]:
info = torch.load(ROOT_PATH + 'My Drive/practica_aidl/backups/BEST_checkpoint_vggsiamese_1.pth.tar')
model = info['model']

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:454: SourceChangeWarning: source code of class 'models.VGGSiameseNet' has changed. you can retrieve the original source code by accessing the object's source attribute or set `torch.nn.Module.dump_patches = True` and use the patch tool to revert the changes.
  warnings.warn(msg, SourceChangeWarning)


In [0]:
query = model.extract_feature(image[None, ...].to(device))
query2 = model.extract_feature(image[None, ...].to(device))

In [0]:
 cos(query, query2)

tensor([1.], device='cuda:0', grad_fn=<DivBackward0>)

In [0]:
feats= None
labels = []
path_dataset = ROOT_PATH + 'My Drive/practica_aidl/backups/examples/test_datasets'

In [0]:
import glob
paths = glob.glob(path_dataset+"/*.jpg")

In [0]:
for path in paths:
  image = Image.open(path).convert('RGB')
  image = trans(image)
  feat = model.extract_feature(image[None, ...].to(device))
  if feats is None:
    feats = feat
  else:
    feats = torch.cat((feats, feat), dim=0)


In [0]:
dist = cos(query, feats)

In [0]:
index_sorted = torch.argsort(dist)
top_10 = reversed(index_sorted[:10])

top = 0
for i in top_10:
    print(str(top) + " - " + str(i.item()) + " - " + str(paths[i]))
    top += 1

0 - 1 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/05.jpg
1 - 2 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/03 (1).jpg
2 - 0 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/03b.jpg
3 - 5 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/07.jpg
4 - 8 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/02.jpg
5 - 3 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/09.jpg
6 - 6 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/01.jpg
7 - 4 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/08.jpg
8 - 7 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/10.jpg


In [0]:
image2 = Image.open(paths[7]).convert('RGB')
image2 = trans(image2)
query3 = model.extract_feature(image2[None, ...].to(device))

In [0]:
dist2 = cos(query3, feats)

In [0]:
print("to search: "+paths[7])
index_sorted = torch.argsort(dist2)
top_10 = reversed(index_sorted[:10])
top = 0
for i in top_10:
    print(str(top) + " - " + str(i.item()) + " - " + str(paths[i]))
    top += 1

to search: /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/10.jpg
0 - 7 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/10.jpg
1 - 4 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/08.jpg
2 - 6 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/01.jpg
3 - 3 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/09.jpg
4 - 8 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/02.jpg
5 - 5 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/07.jpg
6 - 0 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/03b.jpg
7 - 1 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/05.jpg
8 - 2 - /content/drive/My Drive/practica_aidl/backups/examples/test_datasets/03 (1).jpg
